<a href="https://colab.research.google.com/github/surylokesh1432/TreeSpeciesClassification/blob/main/Treespecificclassify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd